https://github.com/dav1ddramb for https://t.me/halftimestat

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import requests
from bs4 import BeautifulSoup
from sklearn.neighbors import KernelDensity
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import os
from functools import reduce

headers = {'User-Agent': 
       ('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko)' 
        'Chrome/47.0.2526.106 Safari/537.36')}

## Defenders with the most G+A in BIG 5 Leagues in the XXI century

In [6]:
FOLDER_PATH = '/Users/daviddramb/Downloads/[HTS]/SCOUTING_FOLDER/'
big_players = pd.DataFrame()

for season_yyyy in range(1999, 2024):
    season_str = '{}-{}-'.format(season_yyyy, season_yyyy+1)

    for league in ['Premier-League', 'La-Liga', 'Bundesliga', 'Serie-A', 'Ligue-1']:
        # ['Champions-League', 'Europa-League', 'Europa-Conference-League']:
        league_players = pd.read_csv(FOLDER_PATH+season_str+league+'_Outfield.csv')
        league_players['season'] = season_yyyy
        league_players['league'] = league

        big_players = pd.concat([big_players, league_players], axis=0)

# big_players.to_csv('BI5_Leagues_players_seasons_XXIcentury.csv')

In [44]:
player_id_fields = ['player', 'birth_year']

# # EITHER any season of players, who were DF/DF, MF, at least, once
# big_defenders = big_players[big_players.position.isin(['DF', 'DF,MF'])][player_id_fields].drop_duplicates()
# # .player.unique() # [big_players['player'].isin(list(big_defenders))]

# big_grouped = big_players.merge(
#     big_defenders, on=player_id_fields, how='inner'
# ).groupby(player_id_fields, as_index=False).sum()


# OR only seasons as a DF/DF, MF
big_grouped = big_players[
    big_players.position.isin(['DF', 'DF,MF'])
].groupby(player_id_fields, as_index=False).sum()

In [86]:
# big_players[
#     big_players.player.str.contains('Dani Alves')
# ] # .sum()[['goals', 'assists']]

In [62]:
exception_players_l = [
    'Zé Roberto', 'Bastian Schweinsteiger', 'Bernd Schneider', 'Bernd Schneider', 'Michel Bastos', 
    'Sejad Salihović', 'Mehdi Mahdavikia', 'Daniele De Rossi', 'James Milner', 'Hasan Salihamidžić', 
    'Gonzalo Castro', 'Matthew Taylor', 'Mathieu Bodmer', '', '', ''
]
display_df = big_grouped[
    ~big_grouped['player'].isin(exception_players_l)
].sort_values('goals_assists', ascending=False).head(15)


### REFORMAT FOR PLOTTING
goals_df = display_df[['player', 'goals',]].rename(columns={'goals': 'value'})
goals_df['type'] = 'goals'

assists_df = display_df[['player', 'assists',]].rename(columns={'assists': 'value'})
assists_df['type'] = 'assists'

display_df = pd.concat([goals_df, assists_df], axis=0)

In [78]:
fig = px.bar(
    display_df, x='value', y='player', color='type', text='value', # x=['goals', 'assists']
    title='Defenders with the most G+A in BIG 5 Leagues in the XXI century', template='plotly_white',
    labels={'player': '', 'value': '', 'type': ''},
    width=900, height=700, # category_orders={'index': display_df.index[::-1]}, 
)
fig.update_traces(textposition='inside')
fig.update_layout(
    font=dict(size=16), yaxis=dict(categoryorder='total ascending'), # showticklabels=False
)
fig.add_annotation(x=0.925, y=0.01, xref='paper', yref='paper', text='@halftimestat', showarrow=False)
fig.show()

In [11]:
big_players.position.unique()

# big_players[big_players.position=='FW,GK'] # DF,GK

array(['DF', 'MF', 'FW', 'FW,MF', 'GK', 'DF,MF', 'DF,FW', '0', 'DF,GK',
       'FW,GK', 'MF,FW', 'MF,DF', 'FW,DF'], dtype=object)

## BIG 5 Leagues goals by players' age

In [17]:
big_players['age_int'] = big_players['age'].apply(lambda x: int(str(x).split('-')[0]))
big_players_season = big_players[big_players['season']==2023]

In [69]:
big_players_season[big_players_season['age_int']==16][['player','position','team','games','goals', 'age', 'minutes']]
big_players[big_players['age_int']>=42][['player','position','team','games','goals', 'age', 'minutes', 'season']]

,player,position,team,games,goals,age,minutes,season
88,Alec Chamberlain,GK,Watford,1.0,0.0,42,1.0,2006
34,Marco Ballotta,GK,Lazio,11.0,0.0,42,865.0,2006
31,Marco Ballotta,GK,Lazio,29.0,0.0,43,2610.0,2007
191,Brad Friedel,GK,Tottenham,1.0,0.0,42,90.0,2013
67,Gianluigi Buffon,GK,Juventus,8.0,0.0,42,720.0,2020
265,Vitorino Hilton,DF,Montpellier,29.0,0.0,42,2117.0,2020


In [49]:
display_df = big_players_season[big_players_season['age_int']>0].groupby('age_int', as_index=False).sum()
display_df = big_players[big_players['age_int']>0].groupby('age_int', as_index=False).sum()

fig = px.bar(
    display_df, x='age_int', y='goals',
    title='BIG 5 Leagues goals by players\' age, XXI century', template='plotly_white',
    labels={'age_int': 'players\' age', 'value': '', 'type': ''},
    width=900, height=500,
)
fig.update_traces(textposition='inside')
fig.update_layout(
    font=dict(size=16),
)
fig.update_traces(marker_color='coral')
fig.add_annotation(x=0.925, y=0.975, xref='paper', yref='paper', text='@halftimestat', showarrow=False)
fig.show()

### Where are the 28yo-29yo attackers?

In [336]:
# big_players['season_flg'] = 1 # a player might've had two clubs in one season, doublecounting, usee count distinct and join
display_df = big_players[(big_players.birth_year>=1994)&(big_players.birth_year<=1995)]\
.groupby(['player', 'birth_year'], as_index=False).sum().sort_values('goals', ascending=False)\
.head(15)# 28yo-29yo

# display_df = big_players[(big_players.birth_year==1998)]\
# .groupby(['player', 'birth_year'], as_index=False).sum().sort_values('goals', ascending=False) # 25yo

In [337]:
big_players[(big_players.birth_year==1998)&(big_players.player.str.contains('Osimhen'))]
big_players[(big_players.player.str.contains('Raheem Sterling'))].shape

(13, 148)

In [338]:
fig = px.bar(
    display_df, x='goals', y='player', color='games', text='goals', # x=['goals', 'assists']
    title='Most goals in BIG 5 Leagues by players born in 1994-1995', template='plotly_white',
    labels={'player': '', 'games': '# of matches<br>in BIG 5 Leagues'},
    width=900, height=700, # category_orders={'index': display_df.index[::-1]}, 
)
fig.update_traces(textposition='inside')
fig.update_layout(
    font=dict(size=16), yaxis=dict(categoryorder='total ascending'), #, showticklabels=False),
)
fig.add_annotation(x=0.925, y=0.01, xref='paper', yref='paper', text='@halftimestat', showarrow=False)
fig.show()

### Most goals per age

#### This season

In [229]:
big_players_season = big_players_season.reset_index(drop=True)

# the index of a player with the most goals per birth_year
most_goals_per_birth_year_idxs = big_players_season.groupby('birth_year', as_index=False)['goals'].idxmax()

# merge the two
big_players_season['goals_index'] = big_players_season.index
big_players_season_most_goals = big_players_season.merge(
    most_goals_per_birth_year_idxs, left_on='goals_index', right_on='goals'
)

# prepare for plotting
display_df = big_players_season_most_goals[
    (big_players_season_most_goals['birth_year_x']>=1984)&(big_players_season_most_goals['birth_year_x']<=2007)
][['player', 'team', 'league', 'birth_year_x', 'goals_x']].sort_values('birth_year_x')
display_df['birth_year'] = display_df['birth_year_x'].astype(str)

In [245]:
fig = px.bar(
    display_df, x='goals_x', y='birth_year', text='player', color='league', # x=['goals', 'assists']
    title='Most goals per age in BIG 5 Leagues, this season', template='plotly_white',
    category_orders={'birth_year': list(display_df['birth_year'])},
    labels={'player': '', 'birth_year': 'birth year', 'goals_x': 'goals', 'league': ''},
    width=900, height=900,
)
fig.update_traces(textposition='outside')
fig.update_layout(
    font=dict(size=16), xaxis=dict(range=[0,1.2*max(display_df['goals_x'])]), # categoryorder='total ascending', 
)
fig.add_annotation(x=0.925, y=0.01, xref='paper', yref='paper', text='@halftimestat', showarrow=False)
fig.show()

In [345]:
big_players_season[big_players_season['birth_year']==1985].sort_values('xg')[['player','birth_year', 'minutes_90s','xg','goals']]

,player,birth_year,minutes_90s,xg,goals
802,Jesús Navas,1985,9.0,0.0,0.0
803,Álvaro Negredo,1985,0.4,0.0,0.0
1729,Guillermo Ochoa,1985,11.0,0.0,0.0
2152,Steve Mandanda,1985,13.0,0.0,0.0
478,Ashley Young,1985,11.0,0.1,0.0
493,Raúl Albiol,1985,10.8,0.4,0.0
782,Luka Modrić,1985,5.6,0.4,0.0


#### XXI century

In [249]:
big_players_grouped = big_players.groupby(['player', 'birth_year'], as_index=False).sum()
big_players_grouped = big_players_grouped.reset_index(drop=True)

# the index of a player with the most goals per birth_year
most_goals_per_birth_year_idxs = big_players_grouped.groupby('birth_year', as_index=False)['goals'].idxmax()

# join the two
big_players_grouped['goals_index'] = big_players_grouped.index
big_players_season_most_goals = big_players_grouped.merge(
    most_goals_per_birth_year_idxs, left_on='goals_index', right_on='goals'
)

# prepare for plotting
display_df = big_players_season_most_goals[
    (big_players_season_most_goals['birth_year_x']>=1970)&(big_players_season_most_goals['birth_year_x']<=2000)
][['player', 'birth_year_x', 'goals_x']].sort_values('birth_year_x')
display_df['birth_year'] = display_df['birth_year_x'].astype(str)

In [255]:
# display_df[['player', 'birth_year_x', 'goals_x']].sort_values('birth_year_x')

In [254]:
fig = px.bar(
    display_df, x='goals_x', y='birth_year', text='player',
    title='Most goals per age in BIG 5 Leagues, XXI century', template='plotly_white',
    category_orders={'birth_year': list(display_df['birth_year'])},
    labels={'player': '', 'birth_year': 'birth year', 'goals_x': 'goals', 'league': ''},
    width=900, height=900,
)
fig.update_traces(textposition='outside')
fig.update_layout(
    font=dict(size=16), xaxis=dict(range=[0,1.35*max(display_df['goals_x'])]), # categoryorder='total ascending', 
)
fig.add_annotation(x=0.925, y=0.01, xref='paper', yref='paper', text='@halftimestat', showarrow=False)
fig.show()

### GKs with the most xG prevented in the past decade/fiveyears

In [276]:
FOLDER_PATH = '/Users/daviddramb/Downloads/[HTS]/SCOUTING_FOLDER/'
big_gks = pd.DataFrame()

for season_yyyy in range(2017, 2024):
    season_str = '{}-{}-'.format(season_yyyy, season_yyyy+1)

    for league in ['Premier-League', 'La-Liga', 'Bundesliga', 'Serie-A', 'Ligue-1']:
        # ['Champions-League', 'Europa-League', 'Europa-Conference-League']:
        league_gks = pd.read_csv(FOLDER_PATH+season_str+league+'_Keepers.csv')
        league_gks['season'] = season_yyyy
        league_gks['league'] = league

        big_gks = pd.concat([big_gks, league_gks], axis=0)

In [294]:
big_gks['league_flg']  = 1
df_filtered = big_gks.groupby(['player', 'birth_year'], as_index=False).sum()
df_filtered['gk_save_pct'] = df_filtered['gk_save_pct'] / df_filtered['league_flg']
df_filtered = df_filtered[
    df_filtered['minutes_90s']>100
]

x_feat = 'gk_psxg_net'
y_feat = 'gk_goals_against'

fig = px.scatter(
    df_filtered, x=x_feat, y=y_feat, custom_data=['player'], color='gk_save_pct',
    size=df_filtered['minutes_90s'] - 0.75*min(df_filtered['minutes_90s']),
    title='BIG 5 League Goalkeepers, 2017-2023', template='none',
    labels={'gk_psxg_net': 'post-shot xG — actual goals allowed',
            'gk_goals_against':'goals allowed',
            'gk_save_pct': 'Save %',}
)
### hover text — add player name
fig.update_traces(
    hovertemplate="<br>".join([
        x_feat+": %{x}",
        y_feat+": %{y}",
        "player: %{customdata[0]}",
    ])
)

######################################################
###### OUTLIERS NEED TO BE SUBSCRIPTED MANUALLY ######
######################################################

### outliers citing
outliers = df_filtered.loc[
    (df_filtered.player.str.contains('Oblak')) |
    (df_filtered.player.str.contains('Alisson')) |
    (df_filtered.player.str.contains('Walter Ben')) |
    (df_filtered.player.str.contains('Neuer')) |
    (df_filtered.player.str.contains('De Gea')) |
    (df_filtered.player.str.contains('Consigli')) |
    (df_filtered.player.str.contains('Ederson')) |
    (df_filtered.player.str.contains('Pickford')) |
    (df_filtered.player.str.contains('ter Stegen')) |
    (df_filtered.player.str.contains('Lloris')) |
    (df_filtered.player.str.contains('Soria')) |
    (df_filtered.player.str.contains('Szcz')) |
    (df_filtered.player.str.contains('Neto')) |
    (df_filtered.player.str.contains('Fabia')) |
    (df_filtered.player.str.contains('Courtois')),
    
    ['player', x_feat, y_feat]
]

### add text to outliers using their (x,y) coordinates:
for p,x,y in outliers.itertuples(index=False):
    x_shift = 0
    y_shift = -20
    if 'Walter Ben' in p or 'Soria' in p or 'Lloris' in p:
        y_shift=20

    fig.add_annotation(
        x=x, y=y,
        text=p,
        showarrow=False,
        xshift=x_shift,
        yshift=y_shift
    )


fig.update_layout(font=dict(size=14))
fig.add_annotation(
    x=0.05, y=0.15, xref='paper', yref='paper',
    text='Circle size indicates <br><b>minutes played by player</b>', 
    showarrow=False, font=dict(family="Courier New, monospace", size=14, color="black"),
    align="center", bgcolor="bisque", opacity=0.75)

fig.add_annotation(x=1, y=1.07, xref='paper', yref='paper', text='@halftimestat', showarrow=False)
fig.show()

In [323]:
# # only GKs with more than <THRESHOLD> (100) matches
# big_gks_matchesthreshold = big_gks.groupby(['player', 'birth_year'], as_index=False).sum()
# big_gks_matchesthreshold = big_gks_matchesthreshold[big_gks_matchesthreshold['minutes_90s']>=100][
#     ['player', 'birth_year']
# ]

# # the mean save_pct
# display_df = big_gks.groupby(['player', 'birth_year'], as_index=False).mean()\
# .sort_values('gk_save_pct', ascending=False)

# # filter the mean save_pct dataset with only players from big_gks_matchesthreshold
# display_df = display_df.merge(big_gks_matchesthreshold, on=['player', 'birth_year'], how='inner').head(15)

# alternative approach
display_df = df_filtered.sort_values('gk_save_pct', ascending=False)[
    ['player','gk_psxg_net','gk_goals_against','gk_save_pct',]
].head(15)

fig = px.bar(
    display_df, x='gk_save_pct', y='player', text='player',
    title='Highest save percentage GKs, 2017-2023', template='plotly_white',
    labels={'player': '', 'birth_year': 'birth year', 'gk_save_pct': 'save %', 'league': ''},
    width=900, height=600,
)
fig.update_traces(textposition='inside')
fig.update_layout(
    font=dict(size=16), yaxis=dict(categoryorder='total ascending', showticklabels=False), 
    xaxis=dict(range=[73.5, 77.5]), 
)
fig.add_annotation(x=0.925, y=0.01, xref='paper', yref='paper', text='@halftimestat', showarrow=False)
fig.show()

In [329]:
display_df = df_filtered.sort_values('gk_save_pct', ascending=False)[
    ['player','gk_psxg_net','gk_goals_against','gk_save_pct',]
].reset_index(drop=True)
display_df[display_df['player'].str.contains('son')]

,player,gk_psxg_net,gk_goals_against,gk_save_pct
3,Alisson,28.2,185.0,76.342857
58,Ederson,-2.9,183.0,70.285714


## Most scoring GKs

In [2]:
players_df = pd.read_csv('BI5_Leagues_players_seasons_XXIcentury.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
gks_df = players_df[players_df['position'].str.contains('GK')]

gks_group_df = gks_df.groupby(['player', 'birth_year'], as_index=False).sum()
gks_group_df.sort_values('goals_assists', ascending=False).head(10)

,player,birth_year,Unnamed: 0,games,games_starts,minutes,minutes_90s,goals,assists,goals_assists,...,xg_net,npxg_net,pass_xa,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct,age_int,season_flg,goals_index
455,Hans-Jörg Butt,1974,193375,320.0,319.0,28571.0,317.4,18.0,1.0,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,370,12,193375
385,François Kamano,1996,41896,23.0,19.0,1485.0,16.5,4.0,3.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,1,41896
904,Paul Robinson,1979,245279,370.0,369.0,33045.0,367.2,1.0,5.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,321,12,245279
831,Nacho González,1971,8896,58.0,58.0,5150.0,57.2,6.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57,2,8896
914,Pepe Reina,1982,804902,651.0,648.0,58274.0,647.5,0.0,5.0,5.0,...,0.0,0.0,0.1,118.0,25.0,0.0,700.0,680,24,804902
71,Alisson,1992,393011,223.0,223.0,20003.0,222.3,1.0,3.0,4.0,...,0.9,0.9,0.0,449.0,66.0,5.0,649.5,190,7,393011
1197,Yassine Bounou,1991,395502,158.0,157.0,14012.0,155.7,1.0,3.0,4.0,...,0.7,0.7,0.1,173.0,28.0,5.0,514.2,203,7,395502
1125,Tim Howard,1979,340713,399.0,398.0,35753.0,397.3,1.0,3.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,390,13,340713
316,Ederson,1993,392306,229.0,229.0,20485.0,227.6,0.0,3.0,3.0,...,0.0,0.0,0.8,321.0,61.0,6.0,643.8,183,7,392306
604,Jussi Jääskeläinen,1975,311411,436.0,435.0,39093.0,434.4,0.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,455,14,311411


## Striker search

In [21]:
FOLDER_PATH = '/Users/daviddramb/Downloads/[HTS]/SCOUTING_FOLDER/'
big_players = pd.DataFrame()

for season_yyyy in range(2023, 2024):
    season_str = '{}-{}-'.format(season_yyyy, season_yyyy+1)

    for league in ['Premier-League', 'La-Liga', 'Bundesliga', 'Serie-A', 'Ligue-1']:
        # ['Champions-League', 'Europa-League', 'Europa-Conference-League']:
        league_players = pd.read_csv(FOLDER_PATH+season_str+league+'_Outfield.csv')
        league_players['season'] = season_yyyy
        league_players['league'] = league

        big_players = pd.concat([big_players, league_players], axis=0)

In [ ]:
big_fw = big_players[big_players['position']=='FW'] # big_players.position.str.contains('FW')
# big_fw = big_fw.groupby(['player', 'birth_year'], as_index=False).sum()

big_fw = big_fw[big_fw['minutes_90s']>5]

big_fw = big_fw[big_fw['npxg_net']>0]
# big_fw = big_fw[big_fw['tackles_att_3rd']/big_fw['minutes_90s']>0.5]
# big_fw = big_fw[big_fw['sca_per90']>1]


big_fw['arbitrary_score'] =\
(big_fw['npxg'] + big_fw['npxg_net'] + big_fw['tackles_att_3rd'] + big_fw['sca']) / big_fw['minutes_90s']

In [2]:
# [HTS] Find best players tool
big_players = pd.read_csv(
    '/Users/daviddramb/Downloads/[HTS]/SCOUTING_FOLDER/2023-multiple_leagues_players_perc_per90.csv'
) # percentiles

In [6]:
big_fw = big_players[big_players['position']=='FW'] # big_players.position.str.contains('FW')
# big_fw = big_fw.groupby(['player', 'birth_year'], as_index=False).sum()

big_fw = big_fw[big_fw['minutes_90s']>5]

# big_fw = big_fw[big_fw['npxg_net']>0]
# big_fw = big_fw[big_fw['tackles_att_3rd']/big_fw['minutes_90s']>0.5]
# big_fw = big_fw[big_fw['sca_per90']>1]
big_fw['arbitrary_score'] = \
big_fw['npxg_per90'] + big_fw['npxg_net_per90'] + big_fw['tackles_att_3rd_per90'] +\
big_fw['progressive_passes_received_per90'] + big_fw['sca_per90']

big_fw[
    [
        'player','team','age','minutes_90s', 'npxg_per90','npxg_net_per90','tackles_att_3rd_per90',
        'progressive_passes_received_per90','sca_per90','arbitrary_score'
    ]
].sort_values('arbitrary_score', ascending=False).head(20)

,player,team,age,minutes_90s,npxg_per90,npxg_net_per90,tackles_att_3rd_per90,progressive_passes_received_per90,sca_per90,arbitrary_score
892,Deniz Undav,Stuttgart,27-180,9.7,0.971014,0.874396,0.946860,0.685990,0.913043,4.391304
255,Mohamed Salah,Liverpool,31-214,19.3,0.821256,0.637681,0.840580,0.946860,0.932367,4.178744
596,Gianluca Scamacca,Atalanta,25-014,7.1,0.797101,0.961353,0.855072,0.913043,0.454106,3.980676
1053,Vinicius Júnior,Real Madrid,23-187,8.5,0.628019,0.772947,0.739130,0.932367,0.879227,3.951691
1413,Kylian Mbappé,Paris S-G,25-026,15.8,0.937198,0.946860,0.140097,0.942029,0.903382,3.869565
131,Son Heung-min,Tottenham,31-191,18.8,0.550725,0.966184,0.608696,0.845411,0.845411,3.816425
1238,Iñaki Williams,Athletic Club,29-214,17.4,0.502415,0.826087,0.714976,0.884058,0.855072,3.782609
828,Gonçalo Paciência,Bochum,29-167,5.3,0.632850,0.917874,0.782609,0.487923,0.927536,3.748792
254,Bukayo Saka,Arsenal,22-132,18.4,0.410628,0.338164,0.980676,0.995169,0.975845,3.700483
689,Kingsley Coman,Bayern Munich,27-216,9.5,0.357488,0.710145,0.681159,0.985507,0.961353,3.695652


In [7]:
big_fw[
    [
        'player','team','age','minutes_90s', 'npxg_per90','npxg_net_per90','tackles_att_3rd_per90',
        'progressive_passes_received_per90','sca_per90','arbitrary_score'
    ]
][big_fw['player']=='Romain Del Castillo']

,player,team,age,minutes_90s,npxg_per90,npxg_net_per90,tackles_att_3rd_per90,progressive_passes_received_per90,sca_per90,arbitrary_score
1309,Romain Del Castillo,Brest,27-292,16.1,0.019324,0.68599,0.772947,0.821256,0.980676,3.280193


In [4]:
[x for x in big_players.columns if 'receiv' in x] # big_fw

['progressive_passes_received_per90', 'passes_received_per90']

## Brentford's defensive excellence

In [97]:
# for
pl_for_season = pd.read_csv(
    '/Users/daviddramb/Downloads/[HTS]/SCOUTING_FOLDER/2023-2024-Premier-League_Teams.csv'
)

# against
pl_ag_season = pd.read_csv(
    '/Users/daviddramb/Downloads/[HTS]/SCOUTING_FOLDER/2023-2024-Premier-League_Teams_VS.csv'
)
pl_ag_season['vs_squad'] = pl_ag_season['squad']
pl_ag_season['squad'] = pl_ag_season['squad'].apply(lambda x: ' '.join(x.split(' ')[1:]))

# unite the for and against dataframes to obtain the diffeence. between the (xG for) and (xG against)
pl_tot_season = pl_for_season.merge(pl_ag_season, on='squad', suffixes=('', '_against'))
pl_tot_season['xg_diff'] = pl_tot_season['xg'] - pl_tot_season['xg_against']
pl_tot_season['goal_diff'] = pl_tot_season['goals'] - pl_tot_season['goals_against']

In [98]:
pl_tot_season[
    ['squad','xg','goals','xg_against','goals_against','xg_diff','goal_diff']
].sort_values('xg_diff', ascending=False)

,squad,xg,goals,xg_against,goals_against,xg_diff,goal_diff
10,Liverpool,44.1,40.0,22.6,17.0,21.5,23.0
12,Manchester City,39.7,47.0,19.1,22.0,20.6,25.0
0,Arsenal,36.4,36.0,16.3,20.0,20.1,16.0
6,Chelsea,41.6,34.0,29.7,31.0,11.9,3.0
1,Aston Villa,37.2,40.0,26.7,25.0,10.5,15.0
3,Brentford,32.6,25.0,25.2,31.0,7.4,-6.0
4,Brighton,34.8,36.0,29.6,31.0,5.2,5.0
14,Newcastle Utd,40.5,41.0,36.5,32.0,4.0,9.0
8,Everton,31.2,24.0,28.9,26.0,2.3,-2.0
17,Tottenham,36.4,41.0,35.5,29.0,0.9,12.0


In [110]:
display_df = pl_tot_season.sort_values('xg_against', ascending=True)

fig = px.scatter(
    display_df, x='xg_against', y='xg', text='squad',
    title='Premier League xG <b>for</b> and <b>against</b>, 23/24', template='plotly_white',
    labels={'age_int': 'players\' age', 'xg': 'Expected Goals For', 'xg_against': 'Expected Goals Against',},
    width=900, height=500,
)
fig.update_layout(
    font=dict(size=14),
)
fig.update_traces(marker_color='coral', textposition='bottom center', marker={'size': 20})
fig.add_annotation(x=0.025, y=0.075, xref='paper', yref='paper', text='@halftimestat', showarrow=False)
fig.show()

## Does Arsenal need a striker?

In [75]:
# players
players_season = pd.read_csv(
    '/Users/daviddramb/Downloads/[HTS]/SCOUTING_FOLDER/2023-2024-Premier-League_Outfield.csv'
)
players_season['goals_xg_diff'] = players_season['goals'] - players_season['xg']

In [96]:
display_df = players_season[
    (players_season['team']=='Arsenal')&(players_season['minutes_90s']>=5)&(players_season['position']!='GK') # (players_season['xg']>0.5)
].sort_values('goals_xg_diff', ascending=False)

fig = px.bar(
    display_df, x='player', y=['goals', 'xg'],
    title='Arsenal players <b>goals</b> against <b>xG</b>, 23/24', template='plotly_white',
    labels={'age_int': 'players\' age', 'player': '', 'variable': '', 'value': ''},
    width=900, height=500, barmode='group', # overlay, relative
)
fig.update_traces(textposition='inside',)
fig.update_layout(
    font=dict(size=16),
)
# fig.update_traces(marker_color='coral')
fig.add_annotation(x=0.025, y=0.975, xref='paper', yref='paper', text='@halftimestat', showarrow=False)
fig.show()

In [94]:
players_season.sort_values('goals_xg_diff', ascending=False)
players_season[players_season['player'].str.contains('land')][
    ['player','team','minutes_90s','goals','xg','npxg','goals_xg_diff']
]

,player,team,minutes_90s,goals,xg,npxg,goals_xg_diff
193,Erling Haaland,Manchester City,14.4,14.0,14.8,11.6,-0.8


In [31]:
[x for x in pl_season.columns if 'xg' in x]
[x for x in pl_ag_season.columns if 'xg' in x]

['xg',
 'npxg',
 'xg_assist',
 'npxg_xg_assist',
 'xg_per90',
 'xg_assist_per90',
 'xg_xg_assist_per90',
 'npxg_per90',
 'npxg_xg_assist_per90',
 'gk_psxg',
 'gk_psnpxg_per_shot_on_target_against',
 'gk_psxg_net',
 'gk_psxg_net_per90',
 'npxg_per_shot',
 'xg_net',
 'npxg_net',
 'xg_assist_net']

## Which league winner had the least points the season after/before?

In [111]:
# league_table = pd.DataFrame()
# for league_id, league_name in [
#     (9, 'Premier-League'), (11, 'Serie-A'), (12, 'La-Liga'), (20, 'Bundesliga'), (13, 'Ligue-1')
# ]:
#     print('\n', league_name)
#     for season_start in range(2000, 2023):
#         print(season_start, end=', ')
#         league_url = 'https://fbref.com/en/comps/{}/{}-{}/{}-{}-{}-Stats'.format(
#             league_id, season_start, season_start+1, season_start, season_start+1, league_name
#         )

#         res = requests.get(league_url, headers=headers)
#         cont = res.content
#         tree = BeautifulSoup(cont, 'html.parser')

#         table = tree.find_all('table')
#         league_table_season = pd.read_html(str(table))[0]
#         league_table_season['league_name'] = league_name
#         league_table_season['season_start'] = season_start

#         league_table = pd.concat([league_table, league_table_season], axis=0)

# league_table.to_csv('league_tables_united_2000-2023.csv')
league_table_united = pd.read_csv('league_tables_united_2000-2023.csv')

In [160]:
league_winners_pts_l = []
for league_name in ['Premier-League', 'Serie-A', 'La-Liga', 'Bundesliga', 'Ligue-1']: # ['Premier-League']: # ['Serie-A']:
    league_table = league_table_united[league_table_united['league_name']==league_name]
    
    for season_start in range(2000, 2022): # league_table['season_start'].unique():
        league_winner_row = league_table[
            (league_table['season_start']==season_start)
            &(league_table['Rk']==1)
        ]
        league_winner_squad = league_winner_row.iloc[0]['Squad']
        league_winner_pts = league_winner_row.iloc[0]['Pts']
        league_winner_rk = league_winner_row.iloc[0]['Rk']
        league_winner_mp = league_winner_row.iloc[0]['MP']

        league_winner_following_row = league_table[
            (league_table['season_start']==season_start+1)
            &(league_table['Squad']==league_winner_squad)
        ]
        league_winner_following_pts = league_winner_following_row.iloc[0]['Pts']
        league_winner_following_rk = league_winner_following_row.iloc[0]['Rk']
        league_winner_following_mp = league_winner_following_row.iloc[0]['MP']

        league_winners_pts_l.append([
            season_start, league_winner_squad, league_winner_pts, league_winner_rk, league_winner_mp,
            league_winner_following_pts, league_winner_following_rk, league_winner_following_mp
        ])

In [161]:
# Turn list into DataFrame
league_winners_pts_df = pd.DataFrame(
    league_winners_pts_l, 
    columns=[
        'season_start','league_winner_squad','league_winner_pts','league_winner_rk','league_winner_mp',
        'league_winner_following_pts','league_winner_following_rk','league_winner_following_mp'
    ]
)

# A bit of feature formatting and generation
league_winners_pts_df['league_winner_pts_game'] = league_winners_pts_df['league_winner_pts'] / league_winners_pts_df['league_winner_mp']
league_winners_pts_df['league_winner_following_pts_game'] = league_winners_pts_df['league_winner_following_pts'] / league_winners_pts_df['league_winner_following_mp']

league_winners_pts_df['season'] = league_winners_pts_df['season_start'].apply(lambda x: str(x)[2:]+'/'+str(x+1)[2:])

# Premier League name shortening for visualization
league_winners_pts_df['league_winner_squad'] = league_winners_pts_df['league_winner_squad'].apply(
    lambda x: 'Man Utd' if x=='Manchester Utd' else 'Man City' if x=='Manchester City' else x
)

In [162]:
league_winners_pts_df.sort_values('league_winner_following_pts_game').head(5)

,season_start,league_winner_squad,league_winner_pts,league_winner_rk,league_winner_mp,league_winner_following_pts,league_winner_following_rk,league_winner_following_mp,league_winner_pts_game,league_winner_following_pts_game,season
15,2015,Leicester City,81,1,38,44,12,38,2.131579,1.157895,15/16
88,2000,Nantes,68,1,34,43,10,34,2.000000,1.264706,00/01
14,2014,Chelsea,87,1,38,50,10,38,2.289474,1.315789,14/15
99,2011,Montpellier,82,1,38,52,9,38,2.157895,1.368421,11/12
108,2020,Lille,83,1,38,55,10,38,2.184211,1.447368,20/21


In [150]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=league_winners_pts_df['season'], y=league_winners_pts_df['league_winner_pts'], 
        name='points in a<br><b>league-winning<br>season</b>', mode="lines+markers+text", 
#         text=league_winners_pts_df['league_winner_squad'], textposition="top center"
    ),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(
        x=league_winners_pts_df['season'], y=league_winners_pts_df['league_winner_following_pts'], 
        name="same team,<br>points in the<br><b>following season</b>", mode="lines+markers+text",
        text=league_winners_pts_df['league_winner_squad'], textposition="top center"
    ),
    secondary_y=False,
)


fig.update_layout(
    title_text='{} winners\' points, 2000-2023'.format(league_name.replace('-',' ')), 
    font=dict(size=14), template='plotly_dark',
)
fig.update_yaxes(title_text='points', secondary_y=False)
fig.add_annotation(x=0.025, y=0.975, xref='paper', yref='paper', text='@halftimestat', showarrow=False)
fig.show()

In [159]:
league_winners_pts_df.corr()
# league_winners_pts_df[~league_winners_pts_df['season_start'].isin([2014, 2015, 2016])].corr()

,season_start,league_winner_pts,league_winner_rk,league_winner_mp,league_winner_following_pts,league_winner_following_rk,league_winner_following_mp,league_winner_pts_game,league_winner_following_pts_game
season_start,1.000000,0.425580,NaN,NaN,-0.103764,0.185093,NaN,0.425580,-0.103764
league_winner_pts,0.425580,1.000000,NaN,NaN,0.298984,-0.301329,NaN,1.000000,0.298984
league_winner_rk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
league_winner_mp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
league_winner_following_pts,-0.103764,0.298984,NaN,NaN,1.000000,-0.922065,NaN,0.298984,1.000000
league_winner_following_rk,0.185093,-0.301329,NaN,NaN,-0.922065,1.000000,NaN,-0.301329,-0.922065
league_winner_following_mp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
league_winner_pts_game,0.425580,1.000000,NaN,NaN,0.298984,-0.301329,NaN,1.000000,0.298984
league_winner_following_pts_game,-0.103764,0.298984,NaN,NaN,1.000000,-0.922065,NaN,0.298984,1.000000


## The connection between ticket prices and match attendance

In [183]:
attendance_df = pd.read_csv('big5_leagues_attendance.csv')
attendance_df['season'] = attendance_df['season_yyyy'].apply(lambda x: str(x)[2:]+'/'+str(x+1)[2:])

# Exclude
attendance_df.loc[attendance_df['season']=='20/21','Average'] = 0

# Attendance percentage
attendance_df['attendance_percentage'] = attendance_df['Average'] / attendance_df['Capacity']

In [217]:
display_df = attendance_df[attendance_df['league_name']!='Ligue 1'] # [attendance_df['league_name']=='Serie A']
display_df = display_df.groupby(['league_name', 'season'], as_index=False).mean()

fig = px.line(
    display_df, x='season', y='attendance_percentage', # ['Average', 'attendance_percentage'],
    color='league_name', 
    title='League matches attendance, 2000-2023', template='plotly_white',
    labels={'attendance_percentage':'share of the stadium filled,<br>season average','league_name':'','season':''},
    width=900, height=500
)

fig.update_layout(
    font=dict(size=16),
)
# fig.update_traces(marker_color='coral')
fig.add_annotation(x=0.025, y=0.075, xref='paper', yref='paper', text='@halftimestat', showarrow=False)
fig.show()

In [224]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

display_df = attendance_df[attendance_df['club_name'].str.contains('Inter')]
fig.add_trace(
    go.Scatter(
        x=display_df['season'], y=display_df['attendance_percentage'], 
        name='<b>Inter Milan</b> attendance %', mode="lines+markers+text", 
    ),
    secondary_y=False,
)

display_df = attendance_df[
    (attendance_df['club_name'].str.contains('Milan'))&(attendance_df['club_name'].str.contains('AC'))
]
fig.add_trace(
    go.Scatter(
        x=display_df['season'], y=display_df['attendance_percentage'], 
        name="<b>AC Milan</b> attendance %", mode="lines+markers+text",
    ),
    secondary_y=False,
)

display_df = attendance_df[attendance_df['club_name'].str.contains('Juve')]
fig.add_trace(
    go.Scatter(
        x=display_df['season'], y=display_df['attendance_percentage'], 
        name="<b>Juventus</b> attendance %", mode="lines+markers+text",
    ),
    secondary_y=False,
)


fig.update_layout(
    title_text='Home match attendance percentage, 2000-2023', 
    font=dict(size=14), template='plotly_white',
)
fig.update_xaxes(
    categoryorder='array', categoryarray=attendance_df[attendance_df['club_name'].str.contains('Inter')]['season']
)
fig.update_yaxes(title_text='share of the stadium filled,<br>season average', secondary_y=False)
fig.add_annotation(x=0.025, y=0.975, xref='paper', yref='paper', text='@halftimestat', showarrow=False)
fig.show()